In [ ]:
# default_exp api

# Api module

> This module use eia API in order to fetch oil data. The only thing you need is to create an 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import urllib.request, urllib.error, json, os, datetime
import pandas as pd

series = {
    "WTI":"PET.RWTC.D",
    "BRENT":"PET.RBRTE.D"
}

class Api:
    "class responsible for fetching data from EIA using your `api_key`"
    def __init__(self, api_key=None):
        "if `api_key` is None then it will automatically read your credentials.json file to find the api_key key"
        if api_key is not None:
            self.api_key = api_key
        else:
            try:
                with open('credentials.json','r') as f:
                    data = json.load(f)
                    self.api_key = data['api_key']
            except:
                print('could not read credentials.json or get api_key value from it')
        
    def _request(self, url):
        "request an EIA endpoint `url` and return a json object"
        try:
            with urllib.request.urlopen(url) as url:
                return json.loads(url.read().decode())
        except urllib.error.URLError as e:
            print(e.reason)
        except json.JSONDecodeError as e:
            print(e)  
            
    def fetch_spot(self, series_id="WTI"):
        "fetch spot price of a specific `series_id` e.g BRENT or WTI and return a pandas DataFrame"
        if series_id not in series.keys():
            raise ValueError('{} not recognized as a valid series id'.format(series_id))
        url = 'http://api.eia.gov/series/?api_key={}&series_id={}'.format(self.api_key, series[series_id])
        data = self._request(url)
        df = pd.DataFrame(data['series'][0]['data'])
        df.columns = ['date','price']
        df.date = df.date.apply(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))
        return df
        

In [ ]:
show_doc(Api.__init__)
show_doc(Api._request)
show_doc(Api.fetch_spot)

<h4 id="Api.__init__" class="doc_header"><code>Api.__init__</code><a href="__main__.py#L12" class="source_link" style="float:right">[source]</a></h4>

> <code>Api.__init__</code>(**`api_key`**=*`None`*)

if `api_key` is None then it will automatically read your credentials.json file to find the api_key key

<h4 id="Api._request" class="doc_header"><code>Api._request</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>Api._request</code>(**`url`**)

request an EIA endpoint `url` and return a json object

<h4 id="Api.fetch_spot" class="doc_header"><code>Api.fetch_spot</code><a href="__main__.py#L34" class="source_link" style="float:right">[source]</a></h4>

> <code>Api.fetch_spot</code>(**`series_id`**=*`'WTI'`*)

fetch spot price of a specific `series_id` e.g BRENT or WTI and return a pandas DataFrame

In [ ]:
api = Api()
wti = api.fetch_spot('WTI')
wti.head()

,date,price
0,2020-06-23,40.40
1,2020-06-22,40.60
2,2020-06-19,39.72
3,2020-06-18,38.79
4,2020-06-17,37.91
